In [ ]:
import tensorflow as tf

# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [ ]:
### load the features file into dictionary d, create tf.example with d and write it into .tfrecord file

d = {}
key_type = {}
with open("siemens_in_waymo_combined - Copy.txt") as f:
    for line in f:
        if 'feature' in line or 'value {' in line:
            pass
        elif 'key' in line:
            current_key = line[line.index("\"") + 1: line.index("\"",10,100):];
            d[current_key] = []
        elif 'int64_list' in line:
            key_type[current_key] = 'int64'
        elif 'float_list' in line:
            key_type[current_key] = 'float'
        elif 'bytes_list' in line:
            key_type[current_key] = 'bytes'
        elif 'value: ' in line:
            val_str = line[line.index(":") + 2: len(line):]
            if val_str[len(val_str)-1]=='\n':
                val_str = val_str[:-1]
            if key_type[current_key] == 'bytes':
                val_str_bytes = bytes(val_str,'utf-8')
                d[current_key].append(val_str_bytes.decode('unicode-escape').encode('utf-8'))
            elif key_type[current_key] == 'int64':
                d[current_key].append(int(val_str))
            elif key_type[current_key] == 'float':
                d[current_key].append(float(val_str))


for key, value in d.items():
    if key_type[key] == 'bytes':
         d[key] = _bytes_feature(value)
    elif key_type[key] == 'int64':
         d[key] = _int64_feature(value)
    elif key_type[key] == 'float':
         d[key] = _float_feature(value)

tf_example = tf.train.Example(features=tf.train.Features(feature=d))

record_file = 'siemens_dataset_combined_waymo/train_Terrain with Buildings LiDAR.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    writer.write(tf_example.SerializeToString())